In [ ]:
from __future__ import print_function, division
import qutip as qt
import numpy as np
import scipy.linalg as sp
import math
import cmath
π = math.pi

In [ ]:
def preprocess(mat, vec):

    if mat.shape[0] != mat.shape[1] or mat.shape[0] != vec.shape[0]:
        raise Exception("Matrix A should be square and b should have a matching dimension!")
        
    if not mat.isherm:
        zero_block = np.zeros(mat.shape)
        mat = qt.Qobj(np.bmat([[zero_block, mat.full()] , 
                               [mat.dag().full(), zero_block]]))
        vec = qt.Qobj(np.hstack([b_init.full().flatten(), zero_block[0]]))
        
    eigenvals, eigenvecs = mat.eigenstates()
  
    ### Normalise b and remember normalisation factor
    if vec.norm() != 1:
        nf = vec.norm()
        vec = vec / nf
    else:
        nf = 1

    return mat, vec, nf

In [ ]:
def qft(N):
    mat = 1 / math.sqrt(N) * qt.Qobj([[cmath.exp(1j * i * j / N)
                                       for i in range(N)] for j in range(N)])
    return mat


def angle(k, t0, C):
    return math.asin(C * t0 /(2 * π * k))


def rot(k, t0, C):
    return np.array([[math.cos(angle(k, t0, C)), math.sin(angle(k, t0, C))],
                     [- math.sin(angle(k, t0, C)), math.cos(angle(k, t0, C))]])


def cRot(N, t0, C):
    mat = np.eye(2)
    for k in range(1, N):
        mat = sp.block_diag(mat, rot(k, t0, C))
    rotationmat = qt.Qobj(mat)
    rotationmat.dims =  [[N, 2], [N, 2]]
    return rotationmat


def inv(Quantobj):
    mat = Quantobj.full()
    matinv = np.linalg.inv(mat)
    invQobj = qt.Qobj(matinv)
    invQobj.dims = Quantobj.dims
    return invQobj

In [ ]:
A_init = qt.Qobj([[2., 1.],[1., 2.]])
b_init = qt.Qobj([[0.2] , [0.7]])
prec = 100 # Choose the number of qubits (precision) to represent eigenvalues in the quantum algorithm

# Produce a Hermitian matrix A of dimension 2^n x 2^n (n integer) with eigenvalues in ]0,1]
# and a unit-length vector b of dimension 2^n out of the original inputs A and b 
A, b, nf1 = preprocess(A_init, b_init)

# Some stuff needed for the classical simulation which the quantum system 
# does generically:
eigenvals, eigenvecs = A.eigenstates()  # The eigendecomposition of A

#Condition number, of use for estimating constants
κ = eigenvals.max() / eigenvals.min()
#Additive error achieved in the estimation of |x>, of use for estimating constants
ϵ = 1

### STEP 1: STATE PREPARATION OF b
[HHL: page 2, column 2, center]

Write b in the eigenbasis of A

In [ ]:
 # Create the matrix that diagonalizes A
diagonalizer = qt.Qobj(np.array([eigenvecs[i].full().T.flatten()
                                 for i in range(len(eigenvals))]))
b = diagonalizer * b

### STEP 2: QUANTUM SIMULATION AND QUANTUM PHASE ESTIMATION
[HHL: page 2, column 2, bottom]

In [ ]:
T = prec
t0 = κ / ϵ    # It should be O(κ/ϵ), whatever that means
ψ0 = qt.Qobj([[math.sqrt(2 / T) * math.sin(π * (τ + 0.5) / T)] 
                for τ in range(T)])

# Order is b, τ, and then ancilla
evo = qt.tensor(qt.identity(A.shape[0]), qt.ket2dm(qt.basis(T, 0))) 

for τ in range(1, T):
    evo += qt.tensor((1j * A * τ * t0 / T).expm(), qt.ket2dm(qt.basis(T, τ)))

ψev = evo * qt.tensor(b, ψ0)

ftrans = qt.tensor(qt.identity(b.shape[0]), qft(T))

ψfourier = ftrans * ψev    # This is Eq. 3 in HHL

### STEP 3-1: Conditional Rotation of Ancilla

In [ ]:
total_state = qt.tensor(ψfourier, qt.basis(2, 0))   # Add ancilla for swapping

C = 1 / κ    # Constant, should be O(1/κ)

# Do conditional rotation only on τ and ancilla
final_state = qt.tensor(qt.qeye(b.shape[0]), cRot(T, t0, C)) * total_state

### STEP 3-2: Un-computation of the eigenvalue register

In [ ]:
### Undo Fourier transform
undo_ftrans = qt.tensor(qt.identity(b.shape[0]), inv(qft(T)), qt.identity(2))
ψev_undone = undo_ftrans * final_state

### Undo evolution
undo_evo = qt.tensor(inv(evo), qt.identity(2))

ψ0_with_eiginfo = undo_evo * ψev_undone

### STEP 3-3: Post-selection on $\left|1\right\rangle$ on the ancilla register
We perform the post selection by projecting onto the desired ancilla state and later tracing out the ancilla and the $\left|\ψ_0\right\rangle$ registers.

In [ ]:
projector = qt.tensor(qt.identity(b.shape[0]), qt.identity(T), qt.ket2dm(qt.basis(2, 1)))
postsel = projector * ψ0_with_eiginfo
prob1 = qt.expect(projector, ψ0_with_eiginfo)
# Trace out ancilla and τ registers, leaving only the b register
finalstate = postsel.ptrace([0]) / prob1
finalstate.eigenenergies()

$\left|finalstate\right\rangle$ is essentially a pure state (it should be if all the process was perfect), so now we just isolate that main part, that is our $\left|x\right\rangle$ (after a transformation if our original A was not Hermitian) in the basis that diagonalizes $A$.

In [ ]:
fsevls, fsevcs = finalstate.eigenstates()
x = math.sqrt(fsevls.max()) * fsevcs[fsevls.argmax()]
x

This state is supposed to be $\left|x\right\rangle=A^{-1}\left|b\right\rangle=\sum_j \beta_j\lambda_j^{-1}\left|u_j\right\rangle$, although I don't understand why it has complex numbers.

IMPORTANT NOTE: If A_init is not Hermitian, the output $\left|x\right\rangle$ has dimension double than the input b_init. It is to be interpreted as the representation of the vector (0, x) in the basis that diagonalizes $C = [[0,\,A], [A^{\dagger},\, 0]]$